In [4]:
from pandasql import sqldf
import pandas as pd

In [5]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",skipinitialspace = True)
print(df.head(2))

   39         State-gov   77516  Bachelors  13       Never-married  \
0  50  Self-emp-not-inc   83311  Bachelors  13  Married-civ-spouse   
1  38           Private  215646    HS-grad   9            Divorced   

        Adm-clerical  Not-in-family  White  Male  2174  0  40  United-States  \
0    Exec-managerial        Husband  White  Male     0  0  13  United-States   
1  Handlers-cleaners  Not-in-family  White  Male     0  0  40  United-States   

   <=50K  
0  <=50K  
1  <=50K  


In [6]:
df.columns.tolist()

['39',
 'State-gov',
 '77516',
 'Bachelors',
 '13',
 'Never-married',
 'Adm-clerical',
 'Not-in-family',
 'White',
 'Male',
 '2174',
 '0',
 '40',
 'United-States',
 '<=50K']

In [7]:
df.loc[-1]=df.columns.tolist()
df.index = df.index+1
df = df.sort_index()
df.columns =['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','50k']
#df.to_csv("C:\\Users\\PACKIJO\\Desktop\\acad\\adult.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,50k
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
print(sqldf("select * from df limit 10", globals()))

  age         workclass  fnlwgt  education education-num  \
0  39         State-gov   77516  Bachelors            13   
1  50  Self-emp-not-inc   83311  Bachelors            13   
2  38           Private  215646    HS-grad             9   
3  53           Private  234721       11th             7   
4  28           Private  338409  Bachelors            13   
5  37           Private  284582    Masters            14   
6  49           Private  160187        9th             5   
7  52  Self-emp-not-inc  209642    HS-grad             9   
8  31           Private   45781    Masters            14   
9  42           Private  159449  Bachelors            13   

          marital-status         occupation   relationship   race     sex  \
0          Never-married       Adm-clerical  Not-in-family  White    Male   
1     Married-civ-spouse    Exec-managerial        Husband  White    Male   
2               Divorced  Handlers-cleaners  Not-in-family  White    Male   
3     Married-civ-spouse  Handl

In [9]:
#Show me the average hours per week of all men who are working in private sector
sqldf("""select avg(df.'hours-per-week') from df where workclass = 'Private' and sex = 'Male' """, globals())

,avg(df.'hours-per-week')
0,42.221226


In [10]:
#Show me the frequency table for education, occupation and relationship, separately
print(sqldf("""select education, count(education) from df group by education"""))
print(sqldf("""select occupation, count(occupation) from df group by occupation"""))
print(sqldf("""select relationship, count(relationship) from df group by relationship"""))

       education  count(education)
0           10th               933
1           11th              1175
2           12th               433
3        1st-4th               168
4        5th-6th               333
5        7th-8th               646
6            9th               514
7     Assoc-acdm              1067
8      Assoc-voc              1382
9      Bachelors              5355
10     Doctorate               413
11       HS-grad             10501
12       Masters              1723
13     Preschool                51
14   Prof-school               576
15  Some-college              7291
           occupation  count(occupation)
0                   ?               1843
1        Adm-clerical               3770
2        Armed-Forces                  9
3        Craft-repair               4099
4     Exec-managerial               4066
5     Farming-fishing                994
6   Handlers-cleaners               1370
7   Machine-op-inspct               2002
8       Other-service               

In [11]:
##4. Are there any people who are married, working in private sector and having a master’s degreey
sqldf("""select count(0) from df where df.'marital-status' != 'Never-married' and workclass = 'Private' and education = 'Masters'""")

,count(0)
0,660


In [12]:
#5. What is the average, minimum and maximum age group for people working in different sectors
sqldf("""select workclass, avg(age), min(age), max(age) from df group by df.workclass""")

,workclass,avg(age),min(age),max(age)
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436055,17,81
8,Without-pay,47.785714,19,72


In [13]:
#6. Calculate age distribution by country
sqldf("""SELECT df.'native-country',SUM(CASE WHEN age < 20 THEN 1 ELSE 20 END) AS [Under 20],
        SUM(CASE WHEN age BETWEEN 21 AND 60 THEN 1 ELSE 0 END) AS [21-60],
        SUM(CASE WHEN age BETWEEN 61 AND 90 THEN 1 ELSE 0 END) AS [61-90]
 FROM df group by df.'native-country'""")


,native-country,Under 20,21-60,61-90
0,?,11375,521,37
1,Cambodia,361,17,1
2,Canada,2344,97,18
3,China,1500,67,8
4,Columbia,1161,54,4
5,Cuba,1900,82,13
6,Dominican-Republic,1381,67,2
7,Ecuador,560,26,2
8,El-Salvador,2006,92,3
9,England,1743,78,8


In [ ]:
sqldf("""select df.'capital-gain',df.'capital-loss',(df.'capital-gain'-df.'capital-loss') as df.'Net-Capital-Gain' from df """)